In [1]:
#Background of character and situation

In [2]:
#Import Libraries

In [4]:
#Import API data + data cleanup

In [5]:
#Full Geo map grouped by suburbs

In [6]:
#Input of house parameters

In [8]:
#Another map to show the options

In [12]:
#Calculators

In [9]:
#A new client story and parameters

In [10]:
#New geo plot that contains house options for client B

In [13]:
#Calculators

In [14]:
#A couple of analysis questions including: If you had unlimited money, where would be the best value for money for an investment etc